In [1]:
pip install fosforml

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install statsmodels

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"MASTER_DATA"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [6]:
df.columns

Index(['PRODUCT_CODE', 'PRODUCT_DESCRIPTION', 'CATEGORY_CODE', 'CATEGORY',
       'MANUFACTURING_LT', 'MFG_COST', 'LABOR_COST', 'PROFIT_MARGIN',
       'CUSTOMER_CODE', 'PRODUCT_ID', 'Production_Cost', 'Profit', 'Sales',
       'SUM_Part_Cost', 'CUSTOMER_NAME', 'CITY', 'STATE_PROVINCE',
       'COUNTRY_CODE', 'TRANSIT_LT', 'ORDER_ID',
       'Fact_Customer_Orders_PRODUCT_ID', 'ORDER_QTY', 'ORDER_DATE',
       'PART_CODE', 'QPA', 'UOM', 'UNIT_PRICE', 'LEAD_TIME_IN_WEEKS', 'MOQ',
       'PROD_LIFE_CYCLE'],
      dtype='object')

In [7]:
data = df[['ORDER_ID','Fact_Customer_Orders_PRODUCT_ID','ORDER_DATE','ORDER_QTY']]

In [8]:
# Convert ORDER_DATE to datetime if it's not already
data['ORDER_DATE'] = pd.to_datetime(data['ORDER_DATE'])

In [9]:
# Feature engineering: Extracting useful features from ORDER_DATE
data['ORDER_MONTH'] = data['ORDER_DATE'].dt.month
data['ORDER_DAY'] = data['ORDER_DATE'].dt.day

In [11]:
# Selecting features and target variable
X = data[['ORDER_MONTH', 'ORDER_DAY', 'Fact_Customer_Orders_PRODUCT_ID']]
y = data['ORDER_QTY']

In [12]:
# Encoding categorical variable PRODUCT_ID
X = pd.get_dummies(X, columns=['Fact_Customer_Orders_PRODUCT_ID'], drop_first=True)

In [16]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Creating and training the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [18]:
# Making predictions on the test set
y_test_pred = model.predict(X_test)

In [19]:
# Creating a DataFrame for the test data and predictions
test_data = X_test.copy()
y_test=test_data['Actual_ORDER_QTY'] 
y_test_pred=test_data['Predicted_ORDER_QTY'] 

KeyError: 'Actual_ORDER_QTY'